In [1]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import scipy.stats
import utils.plot
import models.train_metaClf
import models.clusterUtils

/home/xzhang/anaconda3/envs/c2p/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
savedir_processed='/data/xzhang/neuro/processed'
resDir='/data/xzhang/neuro/results/plots/cnnvaeexp0_segNucall'
    
with open(os.path.join(savedir_processed,'meta_processed'), 'rb') as output:
    meta=pickle.load(output)

# annoUsed=np.array(['AAO', 'AAD', 'Duration', 'Gender','Clinical Diag', 'Path Diag','Brain Weight', 'Mutations',
#                    'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43'])
annoUsed=np.array(['Path Diag','Clinical Diag','Brain Weight', 'Mutations',
                   'APOE', 'Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD','alpha-syn', 'TDP43',
                   'AAO', 'AAD', 'Duration', 'Gender'])
annoUsed_numeric=np.array(['AAO', 'AAD', 'Duration','Brain Weight','Braak Tau', 'Thal Phase', 'CERAD', 'CAA', 'SVD'])

meta=meta[annoUsed]
stains=np.array(['DAPI','GFAP','MAP2','LMNB'])

In [3]:
with open(os.path.join(savedir_processed,'allPatientID'), 'rb') as output:
    allPatientIDs=pickle.load(output)
meta=meta.loc[np.unique(allPatientIDs)]

In [ ]:
imgStats_clf=pd.read_csv(os.path.join(savedir_processed,'imgStats_clf.csv'),index_col=0)
imgStats_clf_pID=np.copy(imgStats_clf.index)
imgStats_clf_pID

In [ ]:
for sidx in range(imgStats_clf_pID.size):
    imgStats_clf_pID[sidx]=imgStats_clf_pID[sidx].split('-')[0]+'-'+imgStats_clf_pID[sidx].split('-')[1]
imgStats_clf_pID

In [ ]:
seeds=[4,8,9,10,11,12]
for anno in annoUsed:
# for anno in ['APOE','alpha-syn', 'TDP43']:
    print(anno)
    regrs=False
#     if anno in annoUsed_numeric:
#         regrs=True
#     if anno not in annoUsed_numeric:
#         continue
        
    
    #construct labels
    
    meta['keep']=True
    if anno not in annoUsed_numeric:
        pathDiag_unique_keep,pathDiag_inverse_keep,pathDiag_counts_keep=np.unique(meta[anno].astype(str),return_counts=True,return_inverse=True)
        pathDiag_unique_keep[pathDiag_counts_keep<=1]='remove'
        pathDiag_unique_keep=pathDiag_unique_keep[pathDiag_inverse_keep]

        meta['keep']=pathDiag_unique_keep!='remove'
    
    meta_allcells=meta.loc[imgStats_clf_pID][anno].to_numpy().astype(str)
    cellIdx=np.logical_and(meta_allcells!='na',meta_allcells!='nan')
    cellIdx=np.logical_and(cellIdx,meta.loc[imgStats_clf_pID]['keep'].to_numpy())
    allCat=meta_allcells[cellIdx]
        
    #get cluster composition
    inputAll=imgStats_clf.to_numpy()[cellIdx]

    if anno not in annoUsed_numeric:
        pathDiag_unique,labels,pathDiag_counts=np.unique(allCat,return_counts=True,return_inverse=True)
        weights=np.sum(pathDiag_counts)/pathDiag_counts
    else:
        pathDiag_unique=None
        weights=None
        labels=allCat.astype(float)
    if labels.size<2:
        continue
    annoName=anno
    if ' ' in anno:
        annoName=anno.split(' ')
        annoName=annoName[0]+'_'+annoName[1]
    for seed in seeds:
        print('seed: ',seed)
        name='exp0_allMetaClf_nucSeg_baselineImgStats_seed'+str(seed)+'_'+annoName
        savepath_curr=os.path.join(resDir,'metadataClf',name)
        if not os.path.exists(os.path.join(resDir,'metadataClf')):
            os.mkdir(os.path.join(resDir,'metadataClf'))
        if not os.path.exists(savepath_curr):
            os.mkdir(savepath_curr)
        logsavepath=os.path.join(savepath_curr,'log')
        modelsavepath=os.path.join(savepath_curr,'model')
        plotsavepath=os.path.join(savepath_curr,'plots')

        if os.path.exists(os.path.join(plotsavepath,'predictions'+str(5900)+'.csv')):
            continue

        if not os.path.exists(logsavepath):
            os.mkdir(logsavepath)
        if not os.path.exists(modelsavepath):
            os.mkdir(modelsavepath)
        if not os.path.exists(plotsavepath):
            os.mkdir(plotsavepath)

        models.train_metaClf.train_metaClf(inputAll, pathDiag_unique, labels, logsavepath, modelsavepath, plotsavepath, imgStats_clf_pID[cellIdx], imgStats_clf.index[cellIdx], np.arange(np.sum(cellIdx)),weights=weights, use_cuda=True, seed=seed, testepoch=3500, epochs=6000, saveFreq=100, lr=0.001, weight_decay=0, batchsize=32, model_str='fc3', fc_dim=256,regrs=regrs)